In [ ]:
import sys
sys.path.append('/home/onyxia/work/libsigma')
import numpy as np
import geopandas as gpd
from osgeo import gdal, ogr
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
import classification as cla
import os

# define parameters
shp_path = '../results/data/sample/Sample_BD_foret_T31TCJ.shp'
raster_path = '../results/data/img_pretraitees/Serie_temp_S2_ndvi.tif'
raster_temp_path = '../results/data/classif/raster_temp.tif'
output_path = '../results/data/classif/carte_essences_echelle_pixel.tif'
field_name = 'Code_pixel'  # field containing the numeric label of the classes

In [ ]:
# for those parameters, you know how to get theses information if you had to
raster = gdal.Open(raster_path)
spatial_resolution = 10
geo_transform = raster.GetGeoTransform()
    
# Calculer les coordonnées xmin, ymin, xmax, ymax
xmin_ref = geo_transform[0]
ymax_ref = geo_transform[3]
xmax_ref = xmin_ref + geo_transform[1] * raster.RasterXSize
ymin_ref = ymax_ref + geo_transform[5] * raster.RasterYSize

# Maintenant, utilisez ces coordonnées dans la commande gdal_rasterize
cmd_pattern = ("gdal_rasterize -a {field_name} "
                "-tr {spatial_resolution} {spatial_resolution} "
                "-te {xmin} {ymin} {xmax} {ymax} -ot Byte -of GTiff "
                "{in_vector} {out_image}")

# Remplir la commande avec les paramètres
cmd = cmd_pattern.format(in_vector=shp_path, xmin=xmin_ref, ymin=ymin_ref, xmax=xmax_ref,
                        ymax=ymax_ref, out_image=raster_temp_path, field_name=field_name,
                        spatial_resolution=spatial_resolution)
                    
os.system(cmd)

0...10...20...30...40...50...60...70...80...90...100 - done.


0

In [ ]:
X, Y, t = cla.get_samples_from_roi(raster_path, raster_temp_path)
print(X.shape)
print(Y.shape)
# split samples
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.5)

/opt/conda/lib/python3.12/site-packages/osgeo/gdal.py:311: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


(2468380, 6)
(2468380, 1)


In [ ]:
rf_classifier = RandomForestClassifier(
    max_depth=50,          # Profondeur maximale de l'arbre
    oob_score=True,        # Utilisation de l'échantillonnage en dehors de la sac (Out-of-Bag)
    max_samples=0.75,      # Utilisation de 75% des échantillons pour chaque arbre
    class_weight='balanced',  # Rééquilibrage des classes
)

# Entraîner le modèle sur les données d'entraînement
rf_classifier.fit(X_train, Y_train)

# Prédire les étiquettes pour l'ensemble de test
Y_pred = rf_classifier.predict(X_test)

# Évaluer la performance du modèle
accuracy = accuracy_score(Y_test, Y_pred)
print(f"Précision du modèle : {accuracy:.2f}")

# Afficher le rapport de classification
print("\nRapport de classification :")
print(classification_report(Y_test, Y_pred))

# Afficher le score OOB (out-of-bag)
print(f"\nScore OOB : {rf_classifier.oob_score_:.2f}")

/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Précision du modèle : 0.91

Rapport de classification :
              precision    recall  f1-score   support

          11       0.78      0.29      0.42     35405
          12       0.94      0.97      0.96   1112751
          13       0.78      0.08      0.15     10986
          14       0.83      0.27      0.41     26932
          21       0.01      0.27      0.01       714
          22       0.81      0.16      0.27      1979
          23       0.83      0.59      0.69     10873
          24       0.70      0.48      0.57     26215
          25       0.70      0.45      0.55      8335

    accuracy                           0.91   1234190
   macro avg       0.71      0.40      0.45   1234190
weighted avg       0.92      0.91      0.91   1234190


Score OOB : 0.91


In [ ]:
n_splits = 5
n_repeats = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

# Variables pour stocker les résultats
accuracies = []
classification_reports = []

for repeat in range(n_repeats):
    print(f"Répetition {repeat+1}/{n_repeats}")
    
    # Initialisation des variables pour chaque répétition
    fold_accuracies = []
    fold_reports = []

    for train_index, test_index in skf.split(X, Y):
        # Obtenez les indices d'entraînement et de test
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        
        # Entraîner le modèle
        rf_classifier.fit(X_train, Y_train)
        
        # Prédictions
        Y_pred = rf_classifier.predict(X_test)
        
        # Calcul des performances pour chaque fold
        fold_accuracies.append(accuracy_score(Y_test, Y_pred))
        fold_reports.append(classification_report(Y_test, Y_pred, output_dict=True))
    
    # Moyenne des performances sur les folds pour cette répétition
    accuracies.append(np.mean(fold_accuracies))
    
    # Moyenne des rapports de classification (précision, rappel, f1) sur les folds
    avg_report = {label: np.mean([fold_report[label]['f1-score'] for fold_report in fold_reports]) 
                  for label in fold_reports[0].keys() if label != 'accuracy'}
    classification_reports.append(avg_report)
    
# Calcul des moyennes sur les répétitions
mean_accuracy = np.mean(accuracies)
std_accuracy = np.std(accuracies)

# Moyenne des f1-scores sur toutes les répétitions
mean_classification_report = {key: np.mean([report[key] for report in classification_reports])
                              for key in classification_reports[0]}

# Affichage des résultats
print(f"Précision moyenne : {mean_accuracy:.2f} ± {std_accuracy:.2f}")
print("\nMoyenne des F1-scores par classe :")
for class_label, f1_score in mean_classification_report.items():
    print(f"{class_label}: {f1_score:.2f}")

Répetition 1/30


/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [4]:
# Étape 1 : Entraîner le modèle sur tous les échantillons disponibles
rf_classifier.fit(X, Y)

# Étape 2 : Charger l'image raster et préparer les données pour prédiction
raster = gdal.Open(raster_path)
cols = raster.RasterXSize
rows = raster.RasterYSize
bands = raster.RasterCount
geo_transform = raster.GetGeoTransform()
projection = raster.GetProjection()

# Lire toutes les bandes en tant que tableau
raster_array = raster.ReadAsArray().astype(np.float32)  # Assurez-vous que les valeurs sont bien des float

# Réorganiser les dimensions pour avoir (nb_pixels, nb_bandes)
data_for_prediction = raster_array.reshape(bands, -1).T

# Étape 3 : Prédire les classes pour chaque pixel
predicted_classes = rf_classifier.predict(data_for_prediction)

# Reshape des prédictions pour correspondre à l'image d'origine
predicted_classes = predicted_classes.reshape(rows, cols)

# Étape 4 : Écrire la carte des essences en sortie
output_path = '../results/data/classif/carte_essences_echelle_pixel.tif'
driver = gdal.GetDriverByName('GTiff')
output_raster = driver.Create(output_path, cols, rows, 1, gdal.GDT_Byte)

# Définir les propriétés spatiales
output_raster.SetGeoTransform(geo_transform)
output_raster.SetProjection(projection)

# Ajouter la bande et écrire les données prédictes
output_band = output_raster.GetRasterBand(1)
output_band.WriteArray(predicted_classes)

# Fixer la valeur de nodata à 0
output_band.SetNoDataValue(0)

# Fermer le fichier
output_band.FlushCache()
output_raster = None

print(f"Carte des essences enregistrée sous : {output_path}")

/opt/conda/lib/python3.12/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Carte des essences enregistrée sous : ../results/data/classif/carte_essences_echelle_pixel.tif
